In [121]:
NUM_DAYS_2_WEEKS=14
TTL_NOTPOSSIBLE=-100
import pandas as pd
import pandas.tools.plotting
import numpy as np
# from scipy import stats
# from sklearn import linear_model

#import datetime as dt
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

from fbprophet import Prophet

import importlib
import sys

%pylab inline
pylab.rcParams['figure.figsize'] = (10, 4)

sys.path.insert(0, '/Users/abertran/Documents/gitRepositories/Steve_notebooks')
from capacityHelpers2 import *

figureSize=(18,5)

dir_name='/Users/abertran/Documents/gitRepositories/Steve_notebooks/SteveBobNotebooks/data/'
#+'*cpu-capacity*'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(dir_name) if isfile(join(dir_name, f))]
onlyfiles = [s for s in onlyfiles if 'cpu-capacity' in s]

Populating the interactive namespace from numpy and matplotlib


/Users/abertran/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [122]:
pod_list=set([i.split('-', 1)[0] for i in onlyfiles])
#Remove ap9,eu4 for now as it does not have a spare node so breaks code
if 'ap9' in pod_list: pod_list.remove('ap9')
if 'eu4' in pod_list: pod_list.remove('eu4')
if 'na41' in pod_list: pod_list.remove('na41')
if 'na21' in pod_list: pod_list.remove('na21')
#Pods below taken out because of error in 90th quantile due to not much data - need to continue processing with that error
if 'eu8' in pod_list: pod_list.remove('eu8')
if 'eu10' in pod_list: pod_list.remove('eu10')
if 'eu12' in pod_list: pod_list.remove('eu12')
if 'eu13' in pod_list: pod_list.remove('eu13')
if 'eu14' in pod_list: pod_list.remove('eu14')
if 'eu15' in pod_list: pod_list.remove('eu15')
pod_list

{'ap0',
 'ap1',
 'ap3',
 'ap4',
 'ap5',
 'ap6',
 'ap7',
 'eu1',
 'eu11',
 'eu5',
 'eu6',
 'eu7',
 'eu9',
 'na24',
 'na29',
 'na3',
 'na30',
 'na31',
 'na32',
 'na33',
 'na34',
 'na35',
 'na37',
 'na38',
 'na39',
 'na40',
 'na42',
 'na43',
 'na44',
 'na45',
 'na46',
 'na47',
 'na48',
 'na49',
 'na50',
 'na51',
 'na52',
 'na53',
 'na6',
 'na7',
 'na8'}

In [123]:
dbCpuCapacity = 50

# variables determined by given pod
pod = pd.read_csv('/Users/abertran/Documents/gitRepositories/Steve_notebooks/SteveBobNotebooks/lookups/podList2.csv')
pod.set_index('pod', inplace=True)

cap_adds = pd.read_csv('/Users/abertran/Documents/gitRepositories/Steve_notebooks/SteveBobNotebooks/lookups/podCapacityAdditions.csv', parse_dates=['capacityAdditionDate'], date_parser=lambda x: pd.to_datetime(x))
cap_adds.set_index('pod', inplace=True)

In [124]:
def prophet_forecast(tsCPU2, holidays, dbCPUchangePointsAsDate,topNodes, IRdate,outdir,testData=None):
    
    rms_test=None
    MAE_test=None
    MAPE_test=None
    
    m = Prophet(yearly_seasonality=False,weekly_seasonality=True,daily_seasonality=False,
                holidays=holidays, changepoints=dbCPUchangePointsAsDate)
    m.fit(tsCPU2)

    future = m.make_future_dataframe(periods=360)
    future['cap'] = 50*topNodes
    forecast = m.predict(future)
    
    forecast.set_index('ds', inplace=True)
    
    mask_future=forecast.index>max(tsCPU2.index)
    mask_model=forecast.index<=max(tsCPU2.index)
    model=forecast.loc[mask_model]
    # One-liner to start the debugger here.
    #from IPython.core.debugger import Tracer; Tracer()()
    future_forecast=forecast.loc[mask_future]
    
    
    if(testData is not None):
        errorDF=pd.merge(future_forecast, testData,  how='left', left_index=True, right_index=True)
        rms_test=np.sqrt(np.mean(np.square(errorDF['yhat']-errorDF['y'])))
        MAE_test=(1/len(errorDF))*np.sum(np.abs(errorDF['yhat']-errorDF['y']))
        MAPE_test=(100/len(errorDF))*np.sum(np.abs(errorDF['yhat']-errorDF['y'])/errorDF['y'])
            
        
    plt.style.use('seaborn-whitegrid')
    fig, ax = plt.subplots(figsize=(18,5))
    
    ax.plot(tsCPU2.index,tsCPU2['y'],c='black')
    if(testData is not None):
        ax.plot(testData.index,testData['y'],c='black')
    ax.plot(model.index,model['yhat'],c='#1f77b4')
    ax.fill_between(model.index, model.yhat_lower, model.yhat_upper, facecolor='#1f77b4', alpha=.5)
    ax.plot(future_forecast.index,future_forecast['yhat'],c='#ff7f0e')
    ax.fill_between(future_forecast.index, future_forecast.yhat_lower, future_forecast.yhat_upper, facecolor='#ff7f0e', alpha=.5)
    ax.set_ylabel('CPU Utilization (%)', fontsize=14)
    ax.set_ylim(0,50*topNodes*1.2)
    ax.set_title(podName , fontsize=20)
    ax.axvline(x=IRdate, c='gray', lw=4, alpha=0.5)
    ax.axhline(y=50*topNodes,c='gray',linestyle='--')
    ax.text(x=(pd.to_datetime(IRdate) - pd.Timedelta('15 days')), y=15, s='IR Date', color='black', fontsize=16, fontweight='bold', alpha=0.8)

    plt.savefig(outdir+'prophet_'+podName+'.pdf',format='pdf')
    plt.close(fig)
    
    forecast['ds'] = forecast.index
    pessimisticTTL_prophet = forecast[forecast['yhat_upper'] >= capacity]['ds'].min()
    optimisticTTL_prophet = forecast[forecast['yhat'] >= capacity]['ds'].min()
    return pessimisticTTL_prophet, optimisticTTL_prophet, future, rms_test, MAE_test, MAPE_test

In [125]:
import rpy2;
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()
base = importr('base')
# do things that generate R warnings
base.warnings()

def f(x):
    return datetime.datetime.utcfromtimestamp(x)
f2 = np.vectorize(f)

def quantile_forecast(tsCPU2, IRdate,podName,future,outdir,testData=None):
    rms_test=None
    MAE_test=None
    MAPE_test=None
    tsQuantile=tsCPU2.copy()
    end_quantile_forecast_date=tsQuantile['ds'].iloc[(len(tsQuantile)-1)]
    start_quantile_forecast_date=end_quantile_forecast_date - timedelta(6*365/12+1)
    if(podName in cap_adds.index):
        cap_add_dates=cap_adds.loc[[podName],'capacityAdditionDate']
        cap_add_date=np.max(cap_add_dates)
        if(cap_add_date>start_quantile_forecast_date):
            start_quantile_forecast_date=cap_add_date
    
    tdelta=end_quantile_forecast_date-start_quantile_forecast_date
    if(tdelta.days<NUM_DAYS_2_WEEKS) :
        pessimisticTTL_quantile=TTL_NOTPOSSIBLE 
        optimisticTTL_quantile=TTL_NOTPOSSIBLE
    else:     
    
        futureQuantile=future.copy()

        mask=(tsQuantile['ds']>=start_quantile_forecast_date) & (tsQuantile['ds'] <= end_quantile_forecast_date)
        tsQuantile=tsQuantile.loc[mask]
        mask=(futureQuantile['ds']>end_quantile_forecast_date) 
        futureQuantile=futureQuantile.loc[mask]

        tsQuantile['indices'] = 1+(tsQuantile['ds']-tsQuantile['ds'].iloc[0]).astype('timedelta64[D]')
        futureQuantile['indices'] = 1+(futureQuantile['ds']-tsQuantile['ds'].iloc[0]).astype('timedelta64[D]')

        end_quantile_forecast_date=tsQuantile['ds'].iloc[(len(tsQuantile)-1)]
        start_quantile_forecast_date=end_quantile_forecast_date - timedelta(6*365/12+1)

        rdf_model_train = pandas2ri.py2ri(tsQuantile[['indices','y']])
        indices_to_forecast_r = pandas2ri.py2ri(futureQuantile[['indices']])

        quantreg = importr('quantreg')

        robjects.r('''
                   med_coeff <- function(train) {
                            library(quantreg)
                            train1.rf <- rq(y ~ indices, data = train, tau=0.5)
                    }
                    ''')
        robjects.r('''
                   ninetyp_coeff <- function(train) {
                            library(quantreg)
                            train1.rf <- rq(y ~ indices, data = train, tau=0.9)
                    }
                    ''')

        #Predict
        robjects.r('''
                   g <- function(model,indices_to_forecast) {
                            pred=predict(model,data.frame(index=indices_to_forecast),interval="confidence")
                            #pred <- as.data.frame(predict(model, test))
                            pred = as.data.frame(pred)

                    }
                    ''')

        use_median=True
        use_equantile=True
        try:
            #Model and forecast for 50percentile (median) quantile
            r_f = robjects.globalenv['med_coeff']
            rf_model=(r_f(rdf_model_train))
            #model_med=dict(zip(rf_model.names, list(rf_model)))
            med_fitted_values=(np.array(rf_model.rx('fitted.values'))).flatten()
            med_coeff=(np.array(rf_model.rx('coefficients'))).flatten()
            r_g = robjects.globalenv['g']
            pred_median=pandas2ri.ri2py(r_g(rf_model,indices_to_forecast_r))
        except:
            use_median=False
            med_coeff=[0,0]


        try:    
            ##Model and forecast for 90percentile quantile
            r_f = robjects.globalenv['ninetyp_coeff']
            rf_model=(r_f(rdf_model_train))
            #model_ninetyp=dict(zip(rf_model.names, list(rf_model)))
            ninetyp_fitted_values=(np.array(rf_model.rx('fitted.values'))).flatten()
            ninetyp_coeff=(np.array(rf_model.rx('coefficients'))).flatten()
            r_g = robjects.globalenv['g']
            pred_ninetyp=pandas2ri.ri2py(r_g(rf_model,indices_to_forecast_r))
        except:
            use_equantile=False
            ninetyp_coeff=[0,0]


        #Set-up the model quantile lines
        x_start_med_line=tsQuantile.iloc[0]['ds']
        x_end_med_line=tsQuantile.iloc[(len(tsQuantile)-1)]['ds']
        y_start_med_line=med_coeff[0]+med_coeff[1]*tsQuantile.iloc[0]['indices']
        y_end_med_line=med_coeff[0]+med_coeff[1]*tsQuantile.iloc[(len(tsQuantile)-1)]['indices']
        y_start_extrap_line=ninetyp_coeff[0]+ninetyp_coeff[1]*tsQuantile.iloc[0]['indices']
        y_end_extrap_line=ninetyp_coeff[0]+ninetyp_coeff[1]*tsQuantile.iloc[(len(tsQuantile)-1)]['indices']

        #Set-up the forecast quantile lines
        x_forecast_start_med_line=futureQuantile.iloc[0]['ds']
        x_forecast_end_med_line=futureQuantile.iloc[(len(futureQuantile)-1)]['ds']
        y_forecast_start_med_line=med_coeff[0]+med_coeff[1]*futureQuantile.iloc[0]['indices']
        y_forecast_end_med_line=med_coeff[0]+med_coeff[1]*futureQuantile.iloc[(len(futureQuantile)-1)]['indices']
        y_forecast_start_extrap_line=ninetyp_coeff[0]+ninetyp_coeff[1]*futureQuantile.iloc[0]['indices']
        y_forecast_end_extrap_line=ninetyp_coeff[0]+ninetyp_coeff[1]*futureQuantile.iloc[(len(futureQuantile)-1)]['indices']

        #Set-up the confidence interval for the forecast quantile lines
        futureQuantile = futureQuantile.reset_index(drop=True)
        if(use_median):
            #50P
            y_forecast_start_extrap_lwr_med=pred_median.iloc[0]['lower']
            y_forecast_start_extrap_hghr_med=pred_median.iloc[0]['higher']
            y_forecast_end_extrap_lwr_med=pred_median.iloc[(len(futureQuantile)-1)]['lower']
            y_forecast_end_extrap_hghr_med=pred_median.iloc[(len(futureQuantile)-1)]['higher']
            
            pred_median = pred_median.reset_index(drop=True)
            
            pred_median['indices']=futureQuantile['indices']
            pred_median['ds']=futureQuantile['ds']
            
        if(use_equantile):    
            #90P
            y_forecast_start_extrap_lwr_ninetyp=pred_ninetyp.iloc[0]['lower']
            y_forecast_start_extrap_hghr_ninetyp=pred_ninetyp.iloc[0]['higher']
            y_forecast_end_extrap_lwr_ninetyp=pred_ninetyp.iloc[(len(futureQuantile)-1)]['lower']
            y_forecast_end_extrap_hghr_ninetyp=pred_ninetyp.iloc[(len(futureQuantile)-1)]['higher']

            pred_ninetyp = pred_ninetyp.reset_index(drop=True)
            
            pred_ninetyp['indices']=futureQuantile['indices']
            pred_ninetyp['ds']=futureQuantile['ds']
        
        if(testData is not None):
            df_temp=testData.copy(deep=True)
            df_temp.set_index('ds',inplace=True)
            #date2 = df_temp.index.values -  pd.to_timedelta(3.5, unit='d')
            df_model_test_equantile=df_temp['y'].groupby(pd.TimeGrouper(freq='W')).quantile(0.9)
            #df_model_test_equantile.reset_index(level=0, inplace=True)
            date2 = df_model_test_equantile.index.values -  pd.to_timedelta(3.5, unit='d')
            date2 = (date2 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
            date2=f2(date2)
            #date2 = datetime.datetime.utcfromtimestamp(date2)
            #date2 = datetime.utcfromtimestamp(date2)
            df_model_test_equantileDF=pd.DataFrame(df_model_test_equantile)
            df_model_test_equantileDF=df_model_test_equantileDF.reset_index(drop=True)
            df_model_test_equantileDF['ds']=date2
            #df_model_test_equantileDF.set_index('ds', inplace=True)
            df_model_test_equantileDF['indices'] = (df_model_test_equantileDF['ds']-(tsQuantile['ds'].iloc[0]).to_datetime()).astype('timedelta64[D]')
            df_model_test_equantileDF['y_pred']=ninetyp_coeff[0]+ninetyp_coeff[1]*df_model_test_equantileDF['indices']    
            rms_test=np.sqrt(np.mean(np.square(df_model_test_equantileDF['y_pred']-df_model_test_equantileDF['y'])))
            MAE_test=(1/len(df_model_test_equantileDF))*np.sum(np.abs(df_model_test_equantileDF['y_pred']-df_model_test_equantileDF['y']))
            MAPE_test=(100/len(df_model_test_equantileDF))*np.sum(np.abs(df_model_test_equantileDF['y_pred']-df_model_test_equantileDF['y'])/df_model_test_equantileDF['y'])
            
        fig, ax = plt.subplots(figsize=figureSize)
        ax.plot(tsQuantile['ds'],tsQuantile['y'],c='#1f77b4')
        if(testData is not None):
            ax.plot(testData.index,testData['y'],c='#1f77b4')
            ax.plot(df_model_test_equantileDF['ds'],df_model_test_equantileDF['y'],c='#8c564b')
            #ax.plot(df_model_test_equantile.index.values[1:],df_model_test_equantile[1:],c='#8c564b')
        if(use_median):
            ax.plot([x_start_med_line,x_end_med_line],[y_start_med_line,y_end_med_line],c='#2ca02c')
            ax.plot([x_forecast_start_med_line,x_forecast_end_med_line],[y_forecast_start_med_line,y_forecast_end_med_line],c='#2ca02c', linestyle='--', dashes=(5, 10), label='_nolegend_')
            #Bounds
            ax.fill_between([x_forecast_start_med_line,x_forecast_end_med_line],[y_forecast_start_extrap_lwr_med,y_forecast_end_extrap_lwr_med],[y_forecast_start_extrap_hghr_med,y_forecast_end_extrap_hghr_med], facecolor='#2ca02c', alpha=0.5)
            
        if(use_equantile):
            ax.plot([x_start_med_line,x_end_med_line],[y_start_extrap_line,y_end_extrap_line],c='#ff7f0e')
            ax.plot([x_forecast_start_med_line,x_forecast_end_med_line],[y_forecast_start_extrap_line,y_forecast_end_extrap_line],c='#ff7f0e', linestyle='--', dashes=(5, 10), label='_nolegend_')
            #Bounds
            ax.fill_between([x_forecast_start_med_line,x_forecast_end_med_line],[y_forecast_start_extrap_lwr_ninetyp,y_forecast_end_extrap_lwr_ninetyp],[y_forecast_start_extrap_hghr_ninetyp,y_forecast_end_extrap_hghr_ninetyp], facecolor='#ff7f0e', alpha=0.5)             
        
        ax.set_title(podName , fontsize=20)
        ax.set_xlabel('Time (d)')
        ax.set_ylabel('CPU Utilization (%)')
        if(use_median & use_equantile):
            ax.legend(['Training data','Median quantile regression line', '90P quantile regression line'])
        elif(use_median):
            ax.legend(['Training data','Median quantile regression line'])
        elif(use_equantile):
            ax.legend(['Training data','90P quantile regression line'])
        ax.axhline(y=50*topNodes,c='gray',linestyle='--')
        ax.axvline(x=IRdate, c='gray', lw=4, alpha=0.5)
        ax.text(x=(pd.to_datetime(IRdate) - pd.Timedelta('15 days')), y=15, s='IR Date', color='black', fontsize=16, fontweight='bold', alpha=0.8)
        ax.grid(True)
        plt.savefig(outdir+'quantile_'+podName+'.pdf',format='pdf')
        plt.close(fig)

        #Determine dates
        if (use_equantile & use_median):
            if(np.any(ninetyp_fitted_values >= capacity)) :
                pessimisticTTL_quantile = tsQuantile['ds'][np.argmax(ninetyp_fitted_values >= capacity)]
                optimisticTTL_quantile=pessimisticTTL_quantile
            else :
                pessimisticTTL_quantile = pred_ninetyp[pred_ninetyp['higher'] >= capacity]['ds'].min()
                optimisticTTL_quantile = pred_ninetyp[pred_ninetyp['fit'] >= capacity]['ds'].min()
        elif(use_equantile):
            if(np.any(ninetyp_fitted_values >= capacity)) :
                pessimisticTTL_quantile = tsQuantile['ds'][np.argmax(ninetyp_fitted_values >= capacity)]
                optimisticTTL_quantile=pessimisticTTL_quantile
            else :
                pessimisticTTL_quantile = pred_ninetyp[pred_ninetyp['higher'] >= capacity]['ds'].min()
                optimisticTTL_quantile = pred_ninetyp[pred_ninetyp['fit'] >= capacity]['ds'].min()
        elif(use_median):
            if(np.any(med_fitted_values >= capacity)) :
                pessimisticTTL_quantile = tsQuantile['ds'][np.argmax(med_fitted_values >= capacity)]
                optimisticTTL_quantile=pessimisticTTL_quantile
            else :
                pessimisticTTL_quantile = pred_median[pred_median['higher'] >= capacity]['ds'].min()
                optimisticTTL_quantile = pred_median[pred_median['fit'] >= capacity]['ds'].min()
        else:
            pessimisticTTL_quantile=TTL_NOTPOSSIBLE 
            optimisticTTL_quantile=TTL_NOTPOSSIBLE
            
        
    
        
    return pessimisticTTL_quantile, optimisticTTL_quantile, rms_test, MAE_test, MAPE_test     

Train/test set-up

In [117]:
outdir='/Users/abertran/Documents/Results/ForecastResults/dbCPUUtil/01222018/'
import csv
import os

try:
    os.remove(outdir+'ForecastResults_dbcpu.csv')
except OSError:
    pass

ttl_labels=["Pod Name", "Prophet Pessimistic TTL", "Prophet optimistic TTL", "Quantile pessimistic TTL", "Quantile optimistic TTL","rms test data prophet", "MAE test data prophet", "MAPE test data prophet","rms test data quantile", "MAE test data quantile", "MAPE test data quantile"]
with open(outdir+'ForecastResults_dbcpu.csv', 'a') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(ttl_labels)

for podName in pod_list:
    print(podName)
    #podName='na32'    

    numNodes = pod.loc[podName]['servers']
    topNodes = numNodes - 1
    capacity = topNodes * dbCpuCapacity

    # various dates to be aware of
    IRdate = pod.loc[podName]['IRdate']

    holidays = pd.DataFrame({
    'holiday': 'holiday',
    'ds': getHolidays(start='20-Nov-2016', end=IRdate).strftime('%d-%b-%Y'),
    'lower_window': -1,
    'upper_window': 1,})

    # fetch data for analysis
    data = importArgusTimeSeriesCSV('/Users/abertran/Documents/gitRepositories/Steve_notebooks/SteveBobNotebooks/data/' + podName + '-cpu-capacity.csv')
    tsCPU2 = pd.Series.to_frame(getRankedTimeSeries3(data, topNodes).sum(axis=1))

    end_train_date=np.max(tsCPU2.index.values) 
    end_train_date=end_train_date -  pd.to_timedelta(60, unit='D')
    end_train_date=(end_train_date - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    end_train_date = datetime.datetime.utcfromtimestamp(end_train_date)
    
    tsCPU2['ds'] = pd.to_datetime(tsCPU2.index.values) 
    mask = (tsCPU2['ds'] <= end_train_date)
    df_model_train = tsCPU2.loc[mask]
    mask = (tsCPU2['ds'] > end_train_date)
    df_model_test = tsCPU2.loc[mask]
    #training_data_length=int(np.round(0.80*tsCPU2.shape[0]))

    #df_model_train = tsCPU2[0:training_data_length]
    #df_model_test=tsCPU2[training_data_length:]

    # dbCPU changepoints and reasons
    cpd = pod.loc[podName]['cpd'].split(',') if not pd.isnull(pod.loc[podName]['cpd']) else None
    cpt = pod.loc[podName]['cpt'].split(',') if not pd.isnull(pod.loc[podName]['cpt']) else None
    cpl = {'Title': cpt, 'changeDate': cpd} if not pd.isnull(pod.loc[podName]['cpd']) else None

    dbCPUchangePoints = pd.DataFrame(data=cpl).reindex() if not pd.isnull(pod.loc[podName]['cpd']) else None
    if dbCPUchangePoints is None:
        dbCPUchangePointsAsDate=pd.to_datetime(cpd)
    else:
        dbCPUchangePoints['changeDate_asDate']=pd.to_datetime(dbCPUchangePoints['changeDate'])
        mask= (dbCPUchangePoints['changeDate_asDate'] >= np.min(df_model_train.index)) & (dbCPUchangePoints['changeDate_asDate'] <= np.max(df_model_train.index))
        dbCPUchangePoints = dbCPUchangePoints.loc[mask]
        dbCPUchangePointsAsDate = dbCPUchangePoints['changeDate_asDate']#pd.to_datetime(cpd)      

    #Prophet
    #df_model_train['ds'] = df_model_train.index
    df_model_train.columns = ['y','ds']
    #df_model_test['ds'] = df_model_test.index
    df_model_test.columns = ['y','ds']

    pessimisticTTL_prophet,optimisticTTL_prophet, future,rms_test_prophet,MAE_test_prophet,MAPE_test_prophet=prophet_forecast(df_model_train, holidays, dbCPUchangePointsAsDate,topNodes, IRdate,outdir,df_model_test)
    pessimisticTTL_quantile,optimisticTTL_quantile,rms_test_quantile,MAE_test_quantile,MAPE_test_quantile=quantile_forecast(df_model_train, IRdate, podName, future,outdir,df_model_test)

    if rms_test_prophet is not None:
        rms_test_prophet=round(rms_test_prophet,2)
    if MAE_test_prophet is not None:
        MAE_test_prophet=round(MAE_test_prophet,2)
    if MAPE_test_prophet is not None:
        MAPE_test_prophet=round(MAPE_test_prophet,2)
    if rms_test_quantile is not None:
        rms_test_quantile=round(rms_test_quantile,2)
    if MAE_test_quantile is not None:
        MAE_test_quantile=round(MAE_test_quantile,2)
    if MAPE_test_quantile is not None:   
        MAPE_test_quantile=round(MAPE_test_quantile,2)
    ttl_times = [podName,pessimisticTTL_prophet, optimisticTTL_prophet, pessimisticTTL_quantile, optimisticTTL_quantile,rms_test_prophet, MAE_test_prophet, MAPE_test_prophet,rms_test_quantile,MAE_test_quantile,MAPE_test_quantile]

    with open(outdir+'ForecastResults_dbcpu.csv', 'a') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(ttl_times)


na50


/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:154: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:72: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()
/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


na8


/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:154: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:72: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()


eu11


...

/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:154: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:72: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()
/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


na52


/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:154: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:72: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()
/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


na40


/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:154: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
/Users/abertran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:72: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()


Current forecast

In [126]:
#Current forecast
import csv
import os

outdir='/Users/abertran/Documents/Results/ForecastResults/dbCPUUtil/01112018/'

try:
    os.remove(outdir+'ForecastResults_dbcpu.csv')
except OSError:
    pass

ttl_labels=["Pod Name", "Prophet Pessimistic TTL", "Prophet optimistic TTL", "Quantile pessimistic TTL", "Quantile optimistic TTL","rms test data prophet", "MAE test data prophet", "MAPE test data prophet","rms test data quantile", "MAE test data quantile", "MAPE test data quantile"]
with open(outdir+'ForecastResults_dbcpu.csv', 'a') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(ttl_labels)

for podName in pod_list:
    print(podName)
#podName='na32'    

    numNodes = pod.loc[podName]['servers']
    topNodes = numNodes - 1
    capacity = topNodes * dbCpuCapacity

    # various dates to be aware of
    IRdate = pod.loc[podName]['IRdate']

    holidays = pd.DataFrame({
    'holiday': 'holiday',
    'ds': getHolidays(start='20-Nov-2016', end=IRdate).strftime('%d-%b-%Y'),
    'lower_window': -1,
    'upper_window': 1,})

    # fetch data for analysis
    data = importArgusTimeSeriesCSV('/Users/abertran/Documents/gitRepositories/Steve_notebooks/SteveBobNotebooks/data/' + podName + '-cpu-capacity.csv')
    tsCPU2 = pd.Series.to_frame(getRankedTimeSeries3(data, topNodes).sum(axis=1))

    # dbCPU changepoints and reasons
    cpd = pod.loc[podName]['cpd'].split(',') if not pd.isnull(pod.loc[podName]['cpd']) else None
    cpt = pod.loc[podName]['cpt'].split(',') if not pd.isnull(pod.loc[podName]['cpt']) else None
    cpl = {'Title': cpt, 'changeDate': cpd} if not pd.isnull(pod.loc[podName]['cpd']) else None

    #dbCPUchangePoints = pd.DataFrame(data=cpl).reindex() if not pd.isnull(pod.loc[podName]['cpd']) else None
    #dbCPUchangePoints['changeDate_asDate']=pd.to_datetime(dbCPUchangePoints['changeDate'])
    #mask=(dbCPUchangePoints['changeDate_asDate'] >= np.min(tsCPU2.index) & dbCPUchangePoints['changeDate_asDate'] <= np.max(tsCPU2.index))
    #dbCPUchangePointsAsDate = dbCPUchangePoints['changeDate_asDate']#pd.to_datetime(cpd)

    dbCPUchangePoints = pd.DataFrame(data=cpl).reindex() if not pd.isnull(pod.loc[podName]['cpd']) else None
    if dbCPUchangePoints is None:
        dbCPUchangePointsAsDate=pd.to_datetime(cpd)
    else:
        dbCPUchangePoints['changeDate_asDate']=pd.to_datetime(dbCPUchangePoints['changeDate'])
        mask= (dbCPUchangePoints['changeDate_asDate'] >= np.min(tsCPU2.index)) & (dbCPUchangePoints['changeDate_asDate'] <= np.max(tsCPU2.index))
        dbCPUchangePoints = dbCPUchangePoints.loc[mask]
        dbCPUchangePointsAsDate = dbCPUchangePoints['changeDate_asDate']#pd.to_datetime(cpd)      

    #Plot time series as-is
    plotTimeSeries(tsCPU2, 
    yAxisLabel='CPU Sum', 
    yLower=None, 
    yUpper=int(capacity)+100,
    chartTitle=podName + ' Host Database Server CPU Utilization Sum (Top '+ str(topNodes) +' Nodes)',
    changePoints=dbCPUchangePoints,
    capacity=capacity, figureName=outdir+'timeseries_'+podName+'.pdf')

    #Prophet
    tsCPU2['ds'] = tsCPU2.index
    tsCPU2.columns = ['y','ds']

    pessimisticTTL_prophet,optimisticTTL_prophet, future,rms_test_prophet,MAE_test_prophet,MAPE_test_prophet=prophet_forecast(tsCPU2, holidays, dbCPUchangePointsAsDate,topNodes, IRdate,outdir)
    pessimisticTTL_quantile,optimisticTTL_quantile,rms_test_quantile,MAE_test_quantile,MAPE_test_quantile=quantile_forecast(tsCPU2, IRdate, podName, future,outdir)
    
    ttl_times = [podName,pessimisticTTL_prophet, optimisticTTL_prophet, pessimisticTTL_quantile, optimisticTTL_quantile,rms_test, MAE_test, MAPE_test]

    with open(outdir+'ForecastResults_dbcpu.csv', 'a') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(ttl_times)
    #Determine dates
      

na50
na8
eu11
na53
ap0
na44
eu9
na30


/Users/abertran/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Error in summary.rq(object, cov = TRUE, ...) : 
  tau + h > 1:  error in summary.rq

  warnings.warn(x, RRuntimeWarning)
/Users/abertran/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: In addition: 
  warnings.warn(x, RRuntimeWarning)
/Users/abertran/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: There were 50 or more warnings (use warnings() to see the first 50)
  warnings.warn(x, RRuntimeWarning)
/Users/abertran/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)


na49
eu6
eu5
na33
na42
ap1
na45
na6
na7
na43
na39
na35
na47
na46
ap5
eu1
na3
na31
na34
ap4
ap6
na48
na51
na37
na29
eu7
ap7
na38
na32
na24
ap3
na52
na40
